## 1. Configuración del Entorno

Primero, montamos Google Drive para acceder a los archivos del proyecto.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

## 2. Preparación de Archivos

Descomprimimos el proyecto. Asegúrate de haber subido `TEL351-PokemonRed.zip` a tu Google Drive.

In [ ]:
import os
import shutil
import zipfile

# Ruta al zip en Drive (ajusta si está en una subcarpeta)
zip_path = '/content/drive/MyDrive/TEL351-PokemonRed.zip'
project_path = '/content/TEL351-PokemonRed'

# Verificación de diagnóstico
if not os.path.exists(zip_path):
    print(f"ERROR: No se encontró el archivo en {zip_path}")
    print("Por favor, asegúrate de haber subido el archivo .zip a la raíz de tu Google Drive.")
    print("Si lo subiste a una carpeta, ajusta la variable 'zip_path'.")
else:
    print(f"Archivo encontrado: {zip_path}")
    file_size_mb = os.path.getsize(zip_path) / (1024*1024)
    print(f"Tamaño del archivo: {file_size_mb:.2f} MB")

    if not zipfile.is_zipfile(zip_path):
        print("ERROR: El archivo no es un ZIP válido.")
        print("Posibles causas:")
        print("1. La subida a Drive no terminó correctamente.")
        print("2. El archivo está corrupto.")
        print("3. Es un acceso directo de Drive y no el archivo real.")
    elif not os.path.exists(project_path):
        print("Descomprimiendo proyecto...")
        try:
            shutil.unpack_archive(zip_path, '/content')
            print("¡Descomprimido exitosamente!")
        except Exception as e:
            print(f"Error al descomprimir: {e}")
    else:
        print("El proyecto ya existe en el entorno.")

if os.path.exists(project_path):
    # Cambiamos al directorio del proyecto
    os.chdir(project_path)
    print(f"Directorio actual: {os.getcwd()}")

## 3. Instalación de Dependencias

Instalamos las librerías necesarias para PyBoy, visualización y RL.

In [ ]:
# Dependencias del sistema para PyBoy y Video
!sudo apt-get update
!sudo apt-get install -y libsdl2-dev libsdl2-image-dev libsdl2-mixer-dev libsdl2-ttf-dev ffmpeg swig

# Dependencias de Python
# Nota: Ignoramos baselines/requirements.txt porque contiene versiones específicas (como ml-dtypes==0.2.0)
# que causan errores de compilación en el entorno moderno de Colab.
# En su lugar, instalamos las versiones compatibles más recientes de las librerías necesarias.

print("Instalando dependencias de Python optimizadas para Colab...")
!pip install --upgrade pip setuptools wheel
!pip install "stable-baselines3[extra]>=2.0.0" \
             "shimmy>=1.1.0" \
             "pyboy>=1.6.9" \
             "gymnasium>=0.28.1" \
             pandas \
             matplotlib \
             opencv-python \
             ipywidgets \
             wandb \
             tensorboard \
             mediapy

print("Dependencias instaladas.")

## 4. Configuración de Visualización en Vivo

Definimos un Callback personalizado para mostrar la pantalla del GameBoy en el notebook mientras entrena.

In [ ]:
import warnings
# Suprimir advertencias no críticas para mantener la salida limpia
warnings.filterwarnings("ignore", message=".*Gym has been unmaintained.*")
warnings.filterwarnings("ignore", category=DeprecationWarning)

import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from stable_baselines3.common.callbacks import BaseCallback
import numpy as np
import cv2

class LiveVisualizerCallback(BaseCallback):
    def __init__(self, check_freq: int = 100, verbose: int = 1):
        super().__init__(verbose)
        self.check_freq = check_freq
        self.image_widget = widgets.Image(format='jpeg')
        display(self.image_widget)

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:
            # Obtener la imagen del entorno
            # Dependiendo del wrapper, la imagen puede estar en diferentes lugares
            # Intentamos renderizar
            try:
                # Accedemos al entorno original de PyBoy si es posible
                # Nota: Esto asume que el entorno soporta render(mode='rgb_array')
                screen = self.training_env.render(mode='rgb_array')
                
                if screen is not None:
                    # Convertir a BGR para OpenCV (si es necesario) o mantener RGB
                    # Codificar a JPEG para el widget
                    _, encoded_image = cv2.imencode('.jpg', cv2.cvtColor(screen, cv2.COLOR_RGB2BGR))
                    self.image_widget.value = encoded_image.tobytes()
            except Exception as e:
                pass # Ignorar errores de renderizado para no detener el entrenamiento
        return True

## 5. Ejecutar Entrenamiento

Aquí configuramos qué agente y qué escenario entrenar. 
Modifica `SCENARIO_ID` y `PHASE_NAME` según `gym_scenarios/scenarios.json`.

In [ ]:
# @title FIX: Parchear red_gym_env_v2.py (PyBoy API Fix)
# Este bloque corrige el error "AttributeError: 'PyBoy' object has no attribute 'screen_buffer'"
# modificando el archivo en el entorno de Colab directamente.

import os
import numpy as np # Asegurar que np esté disponible para el parche si fuera necesario (aunque el parche usa el np del archivo)

# Asegurar que project_path esté definido (por si se ejecuta esta celda aislada)
if 'project_path' not in locals():
    project_path = '/content/TEL351-PokemonRed'

file_path = os.path.join(project_path, 'v2/red_gym_env_v2.py')

if os.path.exists(file_path):
    with open(file_path, 'r') as f:
        content = f.read()
    
    # La línea que causa el error
    old_string = "game_pixels_render = self.pyboy.screen_buffer()  # Returns ndarray (144, 160, 3)"
    
    # La implementación corregida que soporta múltiples versiones de PyBoy
    new_string = """# Patched render retrieval
        if hasattr(self.pyboy, 'screen') and hasattr(self.pyboy.screen, 'ndarray'):
            game_pixels_render = self.pyboy.screen.ndarray
        elif hasattr(self.pyboy, 'botsupport_manager'):
            game_pixels_render = self.pyboy.botsupport_manager().screen().screen_ndarray()
        else:
            game_pixels_render = self.pyboy.screen_image()
            if not isinstance(game_pixels_render, np.ndarray):
                game_pixels_render = np.array(game_pixels_render)"""
    
    if old_string in content:
        content = content.replace(old_string, new_string)
        with open(file_path, 'w') as f:
            f.write(content)
        print(f"✅ Archivo parcheado exitosamente: {file_path}")
    elif "Patched render retrieval" in content:
        print(f"ℹ️ El archivo ya estaba parcheado: {file_path}")
    else:
        print(f"⚠️ No se encontró la línea a parchear en {file_path}. Verifica el contenido manualmente.")
else:
    print(f"❌ Archivo no encontrado: {file_path}")

In [ ]:
import sys
import os
import json
import shutil
import types
import importlib
from typing import Dict, Iterable, List, Optional

from gymnasium import spaces

# Configuración robusta de rutas
project_path = '/content/TEL351-PokemonRed'
if project_path not in sys.path:
    sys.path.append(project_path)

baselines_path = os.path.join(project_path, 'baselines')
if baselines_path not in sys.path:
    sys.path.append(baselines_path)

if os.path.exists(project_path) and os.getcwd() != project_path:
    os.chdir(project_path)
    print(f"Directorio de trabajo establecido en: {os.getcwd()}")

# --- RELOAD MODULES TO APPLY PATCHES ---
def reload_modules() -> None:
    modules_to_reload = [
        'v2.red_gym_env_v2',
        'advanced_agents.features',
        'advanced_agents.wrappers',
        'advanced_agents.base',
        'advanced_agents.train_agents',
        'advanced_agents.combat_apex_agent',
        'advanced_agents.puzzle_speed_agent',
        'advanced_agents.hybrid_sage_agent',
        'advanced_agents.transition_models'
    ]
    for mod_name in modules_to_reload:
        if mod_name in sys.modules:
            try:
                importlib.reload(sys.modules[mod_name])
                print(f"♻️ Recargado: {mod_name}")
            except Exception as exc:
                print(f"⚠️ No se pudo recargar {mod_name}: {exc}")

reload_modules()

# FIX: Copiar events.json a la raíz si no existe, ya que el entorno lo busca ahí
events_source = os.path.join(project_path, 'baselines', 'events.json')
events_dest = os.path.join(project_path, 'events.json')
if os.path.exists(events_source) and not os.path.exists(events_dest):
    shutil.copy(events_source, events_dest)
    print(f"Copiado events.json a {events_dest}")
elif not os.path.exists(events_source):
    print(f"ADVERTENCIA: No se encontró {events_source}")

try:
    from advanced_agents.train_agents import _base_env_config
    from advanced_agents.combat_apex_agent import CombatApexAgent, CombatAgentConfig
    from advanced_agents.puzzle_speed_agent import PuzzleSpeedAgent, PuzzleAgentConfig
    from advanced_agents.hybrid_sage_agent import HybridSageAgent, HybridAgentConfig
except ImportError as exc:
    print("ERROR CRÍTICO: Fallo en imports.")
    raise exc

# --- CARGAR ESCENARIOS ---
with open('gym_scenarios/scenarios.json', 'r') as f:
    scenarios_data = json.load(f)

SCENARIOS: Dict[str, Dict] = {scenario['id']: scenario for scenario in scenarios_data['scenarios']}

AGENT_REGISTRY = {
    'combat': {
        'agent_cls': CombatApexAgent,
        'config_cls': CombatAgentConfig,
        'default_phase': 'battle'
    },
    'puzzle': {
        'agent_cls': PuzzleSpeedAgent,
        'config_cls': PuzzleAgentConfig,
        'default_phase': 'puzzle'
    },
    'hybrid': {
        'agent_cls': HybridSageAgent,
        'config_cls': HybridAgentConfig,
        'default_phase': 'battle'
    }
}

MODELS_DIR = os.path.join(project_path, 'colab_models')
os.makedirs(MODELS_DIR, exist_ok=True)

def resolve_phase(scenario_id: str, phase_name: Optional[str]) -> Dict:
    scenario = SCENARIOS.get(scenario_id)
    if scenario is None:
        raise ValueError(f"Escenario {scenario_id} no encontrado en scenarios.json")
    if phase_name is None:
        raise ValueError("phase_name no puede ser None en Colab; proporciona una fase explícita")
    selected_phase = next((p for p in scenario['phases'] if p['name'] == phase_name), None)
    if selected_phase is None:
        raise ValueError(f"Fase {phase_name} no encontrada en el escenario {scenario_id}")
    return selected_phase

def ensure_state_file(state_file_path: str) -> str:
    if not os.path.exists(state_file_path):
        raise FileNotFoundError(
            f"No se encontró el archivo de estado requerido: {state_file_path}. "
            "Asegúrate de haber subido los .state correctos a gym_scenarios/state_files/."
        )
    return state_file_path

def build_env_overrides(state_file_path: str, headless: bool) -> Dict:
    return {
        'init_state': state_file_path,
        'headless': headless,
        'save_video': False,
        'gb_path': 'PokemonRed.gb',
        'session_path': os.path.join(project_path, 'sessions', f"{os.path.basename(state_file_path)}"),
        'render_mode': 'rgb_array' if headless else 'human',
        'fast_video': headless
    }

def _patch_callbacks(agent, additional_callbacks: Optional[List] = None):
    base_callbacks_method = agent.extra_callbacks

    def _patched_callbacks(self):
        callbacks = list(base_callbacks_method())
        if additional_callbacks:
            callbacks.extend(additional_callbacks)
        return callbacks

    agent.extra_callbacks = types.MethodType(_patched_callbacks, agent)

def train_single_run(
    agent_key: str,
    scenario_id: str,
    phase_name: str,
    total_timesteps: int = 200_000,
    headless: bool = True,
    additional_callbacks: Optional[List] = None
):
    registry_entry = AGENT_REGISTRY.get(agent_key)
    if registry_entry is None:
        raise ValueError(f"Agente desconocido: {agent_key}")

    phase = resolve_phase(scenario_id, phase_name)
    state_file_path = ensure_state_file(phase['state_file'])

    env_overrides = build_env_overrides(state_file_path, headless=headless)
    config = registry_entry['config_cls'](
        env_config=_base_env_config(env_overrides),
        total_timesteps=total_timesteps
    )

    agent = registry_entry['agent_cls'](config)

    env_for_check = agent.make_env()
    obs_space = getattr(env_for_check, 'observation_space', None)
    if isinstance(obs_space, spaces.Dict):
        print("Observación tipo Dict detectada -> cambiando policy a MultiInputPolicy")
        agent.policy_name = types.MethodType(lambda self: "MultiInputPolicy", agent)
    env_for_check.close()

    if additional_callbacks:
        _patch_callbacks(agent, additional_callbacks)

    print(
        f"\n=== Entrenando {agent_key.upper()} en {scenario_id} ({phase_name}) por {total_timesteps:,} pasos ===")
    runtime = agent.train()

    agent_dir = os.path.join(MODELS_DIR, agent_key)
    os.makedirs(agent_dir, exist_ok=True)
    model_path = os.path.join(agent_dir, f"{scenario_id}_{phase_name}.zip")
    runtime.model.save(model_path)
    print(f"Modelo guardado en {model_path}")

    return runtime

def train_plan(
    agent_key: str,
    plan: List[Dict],
    default_timesteps: int = 200_000,
    headless: bool = True,
    callback_factory: Optional[callable] = None
) -> Dict[tuple, object]:
    results = {}
    total_runs = len(plan)
    for run_idx, entry in enumerate(plan, start=1):
        scenario_id = entry['scenario']
        phase_name = entry.get('phase') or AGENT_REGISTRY[agent_key]['default_phase']
        run_timesteps = entry.get('timesteps', default_timesteps)
        callbacks = None
        if callback_factory is not None:
            callbacks = callback_factory(entry)
        print(f"\n>>> [{agent_key.upper()}] Ejecución {run_idx}/{total_runs}")
        runtime = train_single_run(
            agent_key=agent_key,
            scenario_id=scenario_id,
            phase_name=phase_name,
            total_timesteps=run_timesteps,
            headless=headless,
            additional_callbacks=callbacks
        )
        results[(scenario_id, phase_name)] = runtime
    return results

### 5.1 Planes de entrenamiento multi-agente

Las funciones anteriores permiten armar planes de entrenamiento para cada agente avanzado.
Configura las listas con los escenarios y fases que quieras cubrir y corre cada bloque de forma independiente.
Recuerda que cada ejecución puede tardar varios minutos, especialmente si acumulas varios escenarios.

In [ ]:
# Configura los escenarios y fases a entrenar para cada agente.
# Añade o edita entradas según los objetivos de tu experimento.
combat_plan = [
    {"scenario": "pewter_brock", "phase": "battle", "timesteps": 200_000},
    # {"scenario": "cerulean_misty", "phase": "battle", "timesteps": 250_000},
]

puzzle_plan = [
    {"scenario": "pewter_brock", "phase": "puzzle", "timesteps": 180_000},
    # {"scenario": "cerulean_misty", "phase": "puzzle", "timesteps": 220_000},
]

hybrid_plan = [
    {"scenario": "pewter_brock", "phase": "battle", "timesteps": 220_000},
    # {"scenario": "vermillion_lt_surge", "phase": "battle", "timesteps": 250_000},
]

DEFAULT_TIMESTEPS = 200_000
DEFAULT_HEADLESS = True

In [ ]:
# Ejecuta este bloque para entrenar al agente de combate en los escenarios definidos.
combat_runs = train_plan(
"combat",
combat_plan,
default_timesteps=DEFAULT_TIMESTEPS,
headless=DEFAULT_HEADLESS
)

In [ ]:
# Entrenamiento para el agente de puzzles.
puzzle_runs = train_plan(
"puzzle",
puzzle_plan,
default_timesteps=DEFAULT_TIMESTEPS,
headless=DEFAULT_HEADLESS
)

In [ ]:
# Entrenamiento para el agente híbrido.
hybrid_runs = train_plan(
"hybrid",
hybrid_plan,
default_timesteps=DEFAULT_TIMESTEPS,
headless=DEFAULT_HEADLESS
)

In [ ]:
# Opcional: respalda todos los modelos generados en Google Drive.
import os
import shutil
from pathlib import Path

local_models_dir = Path(project_path) / 'colab_models'
drive_backup_dir = Path('/content/drive/MyDrive/PokemonRed_Models_Colab')

if not local_models_dir.exists():
    print("Aún no se han generado modelos en 'colab_models/'. Ejecuta primero los entrenamientos.")
else:
    drive_backup_dir.mkdir(parents=True, exist_ok=True)
    archive_path = shutil.make_archive(str(drive_backup_dir / 'colab_models_backup'), 'zip', local_models_dir)
    print(f"Modelos comprimidos y copiados en: {archive_path}")